### Code used for computing the Natural Baseline Level of groundwater compounds


#### This code reads several time-series of groudwater compounds, and compute the NBL range,

##### The NBL illustrates what thresholds are likely signs of anthropogenic effect by indicating how high or low a parameter's value would be expected under natural conditions (Moreau and Daughney, 2021). 

#### The NBL is computed as the average main descriptors of the wells with NO3 concentrations < 10 mg/L. The maximum NBL is defined as the 90th percentile concentration of the the average final values. This methdology assumes that wells with NO3 concentrations below 10 mg/L can be considered as in the aquifer natural conditions as shown in Lions et al. (2021). 


#### The dataset needed is a simple table with Code, Lat and Lon of your wells to be used in a XLSX file, and a several XLSX files for each available compound. Save the name of each XLSX file as for example: Nitrates = "NO3.xlsx", and place them in a individual folder. 

References: Moreau, M., & Daughney, C. 2021). Defining natural baselines for rates of change in New Zealand’s 
groundwater quality: Dealing with incomplete or disparate datasets, accounting for impacted sites, and 
merging into state of the-environment reporting. Science of The Total Environment, 755, 1–17. 
https://doi.org/10.1016/J.SCITOTENV.2020.143292

Lions, J., Devau, N., Elster, D., Voutchkova, D. D., Hansen, B., Schullehner, J., Petrović Pantić, T., Samolov, 
K. A., Camps, V., Arnó, G., Herms, I., Rman, N., Cerar, S., Grima, J., Giménez-Forcada, E., LuqueEspinar, J. A., Malcuit, E., & Gourcy, L. (2021). A Broad-Scale Method for Estimating Natural 
Background Levels of Dissolved Components in Groundwater Based on Lithology and Anthropogenic 
Pressure. Water, 13(1531). https://doi.org/10.3390/W13111531

Code written by: Thiago Victor Medeiros do Nascimento

In [38]:
import glob, os
import pandas as pd
import numpy as np

At this part the wells information, such as code, latitute and longitute coodidinates, are retrieved from a XLSX file: 

In [88]:
wells_gauges_info=pd.read_excel(r'C:\Users\User\OneDrive\IST\RESEARCH\3_clusteringqualitydata\python\wells-infos.xlsx')
wells_gauges_info.set_index('CODIGO', inplace=True)
wells_gauges_info.head()

,NOME,DISTRITO,CONCELHO,FREGUESIA,BACIA,ALTITUDE (,COORD_X,COORD_Y,SISTEMA AQ,ESTADO,...,st_area_sh_2,st_length__2,COD,Lat,Lon,Num_data,Num_data_2,Num_data_1,xcoord,ycoord
CODIGO,,,,,,,,,,,,,,,,,,,,,
509/17,AC 2,BEJA,FERREIRA DO ALENTEJO,PEROGUARDA,SADO,140,205750,123760,A9 - GABROS DE BEJA,NaN,...,0.035637,1.519198,509/17,123760,205750,19,4,7,-8.067555,38.080698
532/136,AC 8,BEJA,SERPA,SERPA (SALVADOR),GUADIANA,200,246468,106234,A9 - GABROS DE BEJA,EM SERVIÇO,...,0.035637,1.519198,532/136,106234,246468,18,3,6,-7.604608,37.921637
521/44,AC10,BEJA,BEJA,BEJA (SANTIAGO MAIOR),SADO,206,218260,116440,A9 - GABROS DE BEJA,NaN,...,0.035637,1.519198,521/44,116440,218260,17,1,5,-7.925157,38.014586
521/445,FURO MONTE DO CURRAL,BEJA,BEJA,BEJA (SANTIAGO MAIOR),SADO,214.17,219127,116982,A9 - GABROS DE BEJA,NaN,...,0.035637,1.519198,521/445,116982,219127,2,0,2,-7.915270,38.019451
520/65,FURO MONTE DO PAÇO,BEJA,FERREIRA DO ALENTEJO,FERREIRA DO ALENTEJO,SADO,169.24,205637,119305,A9 - GABROS DE BEJA,NaN,...,0.035637,1.519198,520/65,119305,205637,2,0,2,-8.068878,38.040563


In [89]:
wells_gauges = pd.DataFrame()
wells_gauges['COD'] = wells_gauges_info.index
wells_gauges.set_index('COD', inplace=True)
wells_gauges["Lat"] = wells_gauges_info["ycoord"]
wells_gauges["Lon"] = wells_gauges_info["xcoord"]
wells_gauges.head()

,Lat,Lon
COD,,
509/17,38.080698,-8.067555
532/136,37.921637,-7.604608
521/44,38.014586,-7.925157
521/445,38.019451,-7.915270
520/65,38.040563,-8.068878


In [108]:
path =r'C:\Users\User\OneDrive\IST\RESEARCH\3_clusteringqualitydata\python\datats'
filenames = glob.glob(path + "/*.xlsx")

namescompounds = [''] * len(filenames)

i = 0
for filename in filenames:
    namecompound = os.path.basename(filename)
    namecompound = namecompound.replace(".xlsx", "")
    namescompounds[i] = namecompound
    i = i + 1

compounds = namescompounds
compounds

['Al',
 'Ca',
 'Cl',
 'EC',
 'EClab',
 'Fe2',
 'HCO3',
 'K',
 'Mg',
 'Mn2',
 'Na',
 'NO2',
 'NO3',
 'P',
 'pH',
 'pHlab',
 'PO',
 'SO4',
 'T']

In [128]:
table90 = pd.DataFrame(index = compounds, columns = wells_gauges.index)
tablemedian = pd.DataFrame(index = compounds, columns = wells_gauges.index)
table90.head()

COD,509/17,532/136,521/44,521/445,520/65,532/124,532/118,521/448,522/251,521/449,...,509/198,521/155,521/156,521/163,521/284,522/204,522/54,532/157,532/38,532/54
Al,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Ca,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Cl,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
EC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
EClab,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [140]:
i = 0

for filename in filenames:
    
    data=pd.read_excel(filename)
    data.set_index('date', inplace=True)
    datamon=data.resample('M').median()
    
    descriptors = datamon.describe(percentiles=[.10, .25, .5, .75, .90, .95])
    
    table90.loc[compounds[i]] = descriptors.loc["90%"]
    tablemedian.loc[compounds[i]] = descriptors.loc["50%"]
    
    i = i+1


tablemedianT = tablemedian.T

table90T = table90.T

In [143]:
table90T.head()

,Al,Ca,Cl,EC,EClab,Fe2,HCO3,K,Mg,Mn2,Na,NO2,NO3,P,pH,pHlab,PO,SO4,T
COD,,,,,,,,,,,,,,,,,,,
509/17,0.0136,81.5,56.1,1082.6,838.0,0.049,299.5,0.388,31.125,0.01,69.31,0.03,51.9,0.01,7.742,7.7,0.01,52.0,19.98
532/136,0.099,93.712,43.5,718.0,822.0,0.0995,344.8,1.0,39.626,0.006,29.6,0.03,51.8,0.044,8.09,7.6,0.044,37.8,20.03
521/44,NaN,86.905,81.6,790.4,783.0,0.111,NaN,0.0796,29.942,0.02,32.601,0.03,86.8,0.052,7.94,7.8,0.052,61.8,19.2
521/445,0.051,86.2,79.52,NaN,NaN,NaN,313.9,0.1521,33.54,NaN,50.72,0.01051,75.68,NaN,7.68,NaN,NaN,72.8,19.16
520/65,0.011,80.98,43.68,NaN,NaN,NaN,352.0,0.4918,35.38,NaN,45.4,0.00862,56.4,NaN,7.96,NaN,NaN,52.86,20.74


Pay attention that the nitrate time-series table MUST be called NO3.xlsx.

In [144]:
indexmedian = tablemedianT["NO3"] <= 10

descriptorsex = pd.DataFrame(data = datamon.loc[:,indexmedian].describe(percentiles=[.10, .25, .5, .75, .90, .95]).T.mean())
tablebaseline = pd.DataFrame(index = descriptorsex.index, columns = compounds)

,Al,Ca,Cl,EC,EClab,Fe2,HCO3,K,Mg,Mn2,Na,NO2,NO3,P,pH,pHlab,PO,SO4,T
count,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
90%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
95%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [145]:
i = 0

for filename in filenames:
    
    data=pd.read_excel(filename)
    data.set_index('date', inplace=True)
    datamon=data.resample('M').median()
    
    descriptors = pd.DataFrame(data = datamon.loc[:,indexmedian].describe(percentiles=[.10, .25, .5, .75, .90, .95]).T.mean())
    
    tablebaseline.loc[:, compounds[i]] = descriptors.values
    
    i = i+1

tablebaseline

,Al,Ca,Cl,EC,EClab,Fe2,HCO3,K,Mg,Mn2,Na,NO2,NO3,P,pH,pHlab,PO,SO4,T
count,1.000,2.000000,4.000000,0.0,0.0,0.0,2.000000,2.000000,2.000000,0.0,2.000000,2.000000,3.000000,0.0,4.00,0.0,0.0,4.00000,4.000000
mean,0.294,40.450000,24.200000,NaN,NaN,NaN,125.500000,1.309000,14.800000,NaN,14.200000,0.089900,4.063333,NaN,8.00,NaN,NaN,18.42500,16.100000
std,NaN,0.494975,5.752101,NaN,NaN,NaN,3.535534,0.864084,5.091169,NaN,0.707107,0.018526,3.232496,NaN,0.20,NaN,NaN,15.70061,5.290243
min,0.294,40.100000,18.900000,NaN,NaN,NaN,123.000000,0.698000,11.200000,NaN,13.700000,0.076800,0.760000,NaN,7.70,NaN,NaN,5.00000,12.800000
10%,0.294,40.170000,19.230000,NaN,NaN,NaN,123.500000,0.820200,11.920000,NaN,13.800000,0.079420,1.450000,NaN,7.82,NaN,NaN,5.00000,13.040000
25%,0.294,40.275000,19.725000,NaN,NaN,NaN,124.250000,1.003500,13.000000,NaN,13.950000,0.083350,2.485000,NaN,8.00,NaN,NaN,5.00000,13.400000
50%,0.294,40.450000,23.450000,NaN,NaN,NaN,125.500000,1.309000,14.800000,NaN,14.200000,0.089900,4.210000,NaN,8.10,NaN,NaN,16.90000,13.800000
75%,0.294,40.625000,27.925000,NaN,NaN,NaN,126.750000,1.614500,16.600000,NaN,14.450000,0.096450,5.715000,NaN,8.10,NaN,NaN,30.32500,16.500000
90%,0.294,40.730000,29.770000,NaN,NaN,NaN,127.500000,1.797800,17.680000,NaN,14.600000,0.100380,6.618000,NaN,8.10,NaN,NaN,33.07000,21.000000
95%,0.294,40.765000,30.385000,NaN,NaN,NaN,127.750000,1.858900,18.040000,NaN,14.650000,0.101690,6.919000,NaN,8.10,NaN,NaN,33.98500,22.500000


In [146]:
tablebaseline.loc["90%"]

Al         0.29400
Ca        40.73000
Cl        29.77000
EC             NaN
EClab          NaN
Fe2            NaN
HCO3     127.50000
K          1.79780
Mg        17.68000
Mn2            NaN
Na        14.60000
NO2        0.10038
NO3        6.61800
P              NaN
pH         8.10000
pHlab          NaN
PO             NaN
SO4       33.07000
T         21.00000
Name: 90%, dtype: float64